In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

from google.colab import drive

In [ ]:

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/labeled.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()


,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [ ]:
df.dropna()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


In [ ]:
# Приводим метки к целочисленному типу для работы модели
df['toxic'] = df['toxic'].astype(int)

In [ ]:
type(df['toxic'])

pandas.core.series.Series

In [ ]:
from sklearn.model_selection import train_test_split

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    df['comment'], df['toxic'], test_size=0.2, random_state=42
)

print('X_train.shape', X_train.shape)
print('y_train.shape', y_train.shape)
print('X_test.shape', X_test.shape)

X_train.shape (11529,)
y_train.shape (11529,)
X_test.shape (2883,)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
# Загружаем токенизатор для русскоязычной модели BERT
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

# Определяем кастомный класс Dataset для PyTorch
class ToxicCommentsDataset(Dataset):
    def __init__(self, comments, toxics, tokenizer, max_length=128):
        self.comments = comments
        self.labels = toxics
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        # Получаем текст комментария и его метку
        comment = str(self.comments.iloc[idx])
        label = self.labels.iloc[idx]

        # Токенизация комментария
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,  # Добавляем специальные токены [CLS] и [SEP]
            max_length=self.max_length,  # Ограничиваем длину текста
            return_token_type_ids=False,
            padding="max_length",  # Заполняем до max_length
            truncation=True,  # Обрезаем длинные комментарии
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# Создаем датасеты и DataLoader'ы для обучения и тестирования
train_dataset = ToxicCommentsDataset(X_train, y_train, tokenizer)
test_dataset = ToxicCommentsDataset(X_test, y_test, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
# Проверяем, доступен ли GPU, и задаем устройство для обучения
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
# Загружаем предобученную модель BERT для классификации
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)
model.to(device)  # Переносим модель на GPU или CPU

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Определяем оптимизатор и функцию потерь
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Функция обучения на одной эпохе
loss_history = []
def train_epoch(model, dataloader, loss_fn, optimizer):
    model.train()  # Переводим модель в режим обучения
    total_loss = 0

    for batch in dataloader:
        # Переносим данные на устройство
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()  # Обнуляем градиенты

        # Прямой проход (forward pass)
        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)  # Вычисляем функцию потерь

        loss.backward()  # Обратное распространение ошибки
        optimizer.step()  # Обновление параметров модели

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
n_epochs = 3
for epoch in range(n_epochs):
    print(f"Epoch {epoch + 1}/{n_epochs}")
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer)
    print(f"Train loss: {train_loss:.4f}")

Epoch 1/3
Train loss: 0.2808
Epoch 2/3
Train loss: 0.1476
Epoch 3/3
Train loss: 0.0831


In [ ]:
from sklearn.metrics import classification_report
# Функция оценки модели
def evaluate(model, dataloader):
    model.eval()  # Переводим модель в режим оценки
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            # Переносим данные на устройство
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Прямой проход
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Возвращаем отчет по метрикам
    return classification_report(all_labels, all_predictions, digits=4)

In [ ]:
# Оценка на тестовой выборке
report = evaluate(model, test_loader)
print(report)


              precision    recall  f1-score   support

           0     0.9606    0.9280    0.9440      1944
           1     0.8607    0.9212    0.8899       939

    accuracy                         0.9258      2883
   macro avg     0.9106    0.9246    0.9170      2883
weighted avg     0.9281    0.9258    0.9264      2883



In [ ]:
from transformers import BertTokenizer

# Путь для сохранения модели и токенизатора
output_dir = "/content/drive/MyDrive/bert_toxic_classifier"

# Создаем директорию
import os
os.makedirs(output_dir, exist_ok=True)

# Сохранение модели и токенизатора
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Модель и токенизатор сохранены в {output_dir}")


Модель и токенизатор сохранены в /content/drive/MyDrive/bert_toxic_classifier


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Копируем модель и токенизатор на Google Drive
!cp -r /content/bert_toxic_classifier /content/drive/MyDrive/
print("Модель скопирована на Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Модель скопирована на Google Drive.


In [1]:
# Очистка метаданных widgets
if 'widgets' in globals() or 'widgets' in locals():
    from IPython.display import display, Javascript
    display(Javascript('''
        const md = JSON.parse(JSON.stringify(IPython.notebook.metadata));
        if (md.widgets && !md.widgets.state) {
            delete md.widgets;
            IPython.notebook.metadata = md;
        }
    '''))